### **Strategy 1: Employing a Frozen Layer Vector of ChemBERTa as additional feature** 
Goal is to predict drug-drug interactions for different relationship types based on the drug features. The features include information about individual drug side effects(mono side effects) and drug-protein interactions.

1. Features (X): Features are represented by drug_feat, which is a concatenation of the PCA-transformed (mono) and frozen embeddings from ChemBERTa as features in a deep neural network (DNN). 
- drug-label features based on PCA transformed mono side effects 10,184. Binary matrix where each row represents the a drug and column is mono side effecs 

2. Target (Y): Targets are represented by train_label_org, val_label_org, and test_label_org.
- For each drug-drug relationship type (out of the n_drugdrug_rel_types types), the target is binary and indicates whether there is an interaction (1) or not (0) between the drugs for that specific relationship type.

****

In [ ]:
pip install azure-ai-ml azure-identity azureml-fsspec mltable scikit-learn matplotlib tensorflow==2.12.0 keras torch transformers scikit-learn keras-tuner 

In [3]:
!pip freeze

absl-py==2.1.0
accelerate==0.33.0
adal==1.2.7
adlfs==2024.7.0
aiofiles==22.1.0
aiohappyeyeballs==2.3.5
aiohttp==3.10.1
aiohttp-cors==0.7.0
aiosignal==1.3.1
aiosqlite==0.20.0
annotated-types==0.7.0
ansicolors==1.1.8
antlr4-python3-runtime==4.13.2
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1726931217849/work
applicationinsights==0.11.10
arch==5.6.0
argcomplete==3.3.0
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1725356557095/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
astroid==3.3.3
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
astunparse==1.6.3
async-lru @ file:///home/conda/feedstock_root/build_artifacts/async-lru_1690563019058/work
async-timeout==4.0.3
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1722977137225/work
a

In [3]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2025-01-23 06:48:50.618194: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-23 06:48:52.672170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-23 06:48:57.284707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-01-23 06:49:14.918301: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations
from IPython.display import display, HTML
import time
import os
from collections import Counter
from collections import defaultdict
from scipy.stats import ks_2samp
import numpy as np
import random
import pandas as pd
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from keras.layers import Dropout, Activation
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import mltable

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

def load_data_from_azureml(ml_client, data_asset_name, version="1"):
    data_asset = ml_client.data.get(data_asset_name, version=version)
    path = {"folder": data_asset.path}
    tbl = mltable.from_delimited_files(paths=[path])
    return tbl.to_pandas_dataframe()

combo_df = load_data_from_azureml(ml_client, "bio-decagon-combo")
mono_df = load_data_from_azureml(ml_client, "bio-decagon-mono")
targets_all_df = load_data_from_azureml(ml_client, "bio-decagon-targets-all")

data_asset = ml_client.data.get("polypharmacycorpus", version="cleaned2024.03.29.211043")
polypharmacycorpus = pd.read_csv(data_asset.path)

data_asset = ml_client.data.get("645-Stitch-drugs-to-smiles", version="cleaned2024.07.08.002203")
smiles_df = pd.read_csv(data_asset.path)
smiles_df

Found the config file in: /config.json


,STITCH,SMILES
0,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...
1,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3
2,CID000005206,C(OC(C(F)(F)F)C(F)(F)F)F
3,CID000009433,CN1C2=C(C(=O)N(C1=O)C)NC=N2.CN1C2=C(C(=O)N(C1=...
4,CID000003929,CC(=O)NCC1CN(C(=O)O1)C2=CC(=C(C=C2)N3CCOCC3)F
...,...,...
640,CID003086258,CCC1=C2C=C3C(=C(C(=CC4=NC(=CC5=NC(=CC(=C1C)N2)...
641,CID009571074,CC1=C(SC=N1)C=CC2=C(N3C(C(C3=O)NC(=O)C(=NOC)C4...
642,CID000004011,CNCCCC12CCC(C3=CC=CC=C31)C4=CC=CC=C24
643,CID000002182,C1C2=C(C=CC(=C2Cl)Cl)N=C3N1CC(=O)N3


In [7]:
#----------------------------------------------------------------------------
# Returns dictionary from combination ID to pair of stitch IDs, 
# dictionary from combination ID to list of polypharmacy side effects, 
# and dictionary from side effects to their names.
from collections import defaultdict

def load_combo_se(df):
    combo2stitch = {}
    combo2se = defaultdict(set)
    se2name = {}

    for index, row in df.iterrows():
        stitch_id1 = row["STITCH 1"]
        stitch_id2 = row["STITCH 2"]
        se = row["Polypharmacy Side Effect"]
        se_name = row["Side Effect Name"]

        combo = stitch_id1 + "_" + stitch_id2
        combo2stitch[combo] = [stitch_id1, stitch_id2]
        combo2se[combo].add(se)
        se2name[se] = se_name

    n_interactions = sum([len(v) for v in combo2se.values()])
    print("Drug combinations: %d Side effects: %d" % (len(combo2stitch), len(se2name)))
    print("Drug-drug interactions: %d" % (n_interactions))

    return combo2stitch, combo2se, se2name

# Load data
combo2stitch, combo2se, se2name = load_combo_se(combo_df)

def load_mono_se(df):
    stitch2se = defaultdict(set)
    se2name = {}

    for index, row in df.iterrows():
        stitch_id = row["STITCH"]
        se = row["Individual Side Effect"]
        se_name = row["Side Effect Name"]
        
        stitch2se[stitch_id].add(se)
        se2name[se] = se_name

    return stitch2se, se2name

stitch2se, se2name_mono = load_mono_se(mono_df)

Drug combinations: 63473 Side effects: 1317
Drug-drug interactions: 4649441


In [8]:
def load_targets(df):
    stitch2proteins_all = defaultdict(set)

    for index, row in df.iterrows():
        stitch_id = row["STITCH"]
        gene = row["Gene"]
        stitch2proteins_all[stitch_id].add(gene)
    return stitch2proteins_all

stitch2proteins_all = load_targets(targets_all_df)

In [9]:
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype("int")

def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [10]:
# common side effects in drug combinations
def get_se_counter(se_map):
    side_effects = []
    for drug in se_map:
        side_effects += list(set(se_map[drug]))
    return Counter(side_effects)
combo_counter = get_se_counter(combo2se)
print("Most common side effects in drug combinations:")
common_se = []
common_se_counts = []
common_se_names = []
for se, count in combo_counter.most_common(964):
    common_se += [se]
    common_se_counts += [count]
    common_se_names += [se2name[se]]
df = pd.DataFrame(data={"Side Effect": common_se, "Frequency in Drug Combos": common_se_counts, "Name": common_se_names})  
display(df)

Most common side effects in drug combinations:


,Side Effect,Frequency in Drug Combos,Name
0,C0020649,28568,arterial pressure NOS decreased
1,C0002871,27006,anaemia
2,C0013404,26037,Difficulty breathing
3,C0027497,25190,nausea
4,C0032285,24430,neumonia
...,...,...,...
959,C0035220,507,neonatal respiratory distress syndrome
960,C0021099,507,impetigo
961,C0009326,504,Collagen disease
962,C1261473,502,sarcoma


In [11]:
val_test_size=0.05 #5% for validation and 5% test
n_drugs=645
n_proteins=8934
n_drugdrug_rel_types=964

#list of drugs 
#lst: list of unique drug names from combo2stitch
lst=[]
for key , value in combo2stitch.items():
  first_name, second_name = map(lambda x: x.strip(), key.split("_"))
  if first_name not in lst:
    lst.append(first_name)
  if second_name not in lst:
    lst.append(second_name)

#list of proteins
# p: list of unique proteins
p=[]
for k,v in stitch2proteins_all.items():
    for i in v:
        if i not in p:
            p.append(i)

#construct drug-protein-adj matrix
# matrix  n_drugs x n_proteins
drug_protein_adj=np.zeros((n_drugs,n_proteins))
for i in range(n_drugs):
    for j in stitch2proteins_all[lst[i]]:
        k=p.index(j)
        drug_protein_adj[i,k]=1

#construct drug-drug-adj matrices for all side effects

drug_drug_adj_list=[]
l=[]
for i in range(n_drugdrug_rel_types):
  print(i)
  mat = np.zeros((n_drugs, n_drugs))
  l.append(df.at[i,"Side Effect"])
  for se in l:
    print(se)
    for d1, d2 in combinations(list(range(n_drugs)), 2):
      if lst[d1]+"_"+lst[d2]  in combo2se:
        if se in combo2se[lst[d1]+"_"+lst[d2]]:
            mat[d1,d2]=mat[d2,d1]=1
  l=[]
  drug_drug_adj_list.append(mat)
drug_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in drug_drug_adj_list]

#select drug pairs for training & validation & testing
#edges: List of positive drug pairs for each relationship type
#edges_false: list of negative drug pairs
#edges_all: shuffled list of positive and negative drug pairs 
edges=[]
for k in range(n_drugdrug_rel_types):
    l=[]
    for i in range(n_drugs):
        for j in range(n_drugs):
            if drug_drug_adj_list[k][i,j]==1:
                l.append([i,j])
    edges.append(l)
edges_false=[]
for k in range(n_drugdrug_rel_types):
    l=[]
    for i in range(n_drugs):
        for j in range(n_drugs):
            if drug_drug_adj_list[k][i,j]==0:
                l.append([i,j])
    edges_false.append(l)
for k in range(n_drugdrug_rel_types):
    np.random.shuffle(edges[k])
    np.random.shuffle(edges_false[k])
for k in range(n_drugdrug_rel_types):
    a=len(edges[k])
    edges_false[k]=edges_false[k][:a]
edges_all=[]
for k in range(n_drugdrug_rel_types):
    edges_all.append(edges[k]+edges_false[k])
for k in range(n_drugdrug_rel_types):
    np.random.shuffle(edges_all[k])
for k in range(n_drugdrug_rel_types):
    a=len(edges[k])
    edges_all[k]=edges_all[k][:a]
val=[]
test=[]
train=[]
for k in range(n_drugdrug_rel_types):
    a=int(np.floor(len(edges_all[k])*val_test_size))
    val.append(edges_all[k][:a])
    test.append(edges_all[k][a:a+a])
    train.append(edges_all[k][a+a:])
    

0
C0020649
1
C0002871
2
C0013404
3
C0027497
4
C0032285
5
C0015672
6
C0030193
7
C0011991
8
C0004093
9
C0042963
10
C0085649
11
C0015967
12
C0008033
13
C0000737
14
C0398353
15
C0008031
16
C0012833
17
C0004604
18
C0018681
19
C0020538
20
C0009676
21
C0011175
22
C0003467
23
C0035078
24
C0043096
25
C0020456
26
C0013604
27
C0003123
28
C0003862
29
C0022660
30
C0442874
31
C0030196
32
C0042029
33
C0010200
34
C0009806
35
C0039231
36
C0043094
37
C0040034
38
C0032227
39
C0243026
40
C0018802
41
C0027051
42
C1145670
43
C0004238
44
C0013144
45
C0015230
46
C0917801
47
C0037763
48
C0038454
49
C0027947
50
C0020625
51
C0231218
52
C0040822
53
C0011168
54
C0007642
55
C0027404
56
C0041657
57
C0085631
58
C0019080
59
C0017168
60
C0002622
61
C0020443
62
C0575081
63
C0085593
64
C0039070
65
C0036572
66
C0006277
67
C0041834
68
C0010054
69
C0020458
70
C0344232
71
C0033774
72
C0497247
73
C0021311
74
C0232492
75
C0013395
76
C0030554
77
C0151904
78
C0037199
79
C0011615
80
C0011849
81
C0020621
82
C0231528
83
C0030312
84

In [ ]:
drug_info_dict = {}

for drug_id in lst:
    matching_rows = polypharmacycorpus[polypharmacycorpus["STITCH 1"] == drug_id]
    if not matching_rows.empty:
        stitch_name_x = matching_rows.iloc[0]["STITCH NAME_x"]
        drug_info_dict[drug_id] = stitch_name_x
    else:
        matching_rows = polypharmacycorpus[polypharmacycorpus["STITCH 2"] == drug_id]
        if not matching_rows.empty:
            stitch_name_y = matching_rows.iloc[0]["STITCH NAME_y"]
            drug_info_dict[drug_id] = stitch_name_y
        else:
            # Handle the case where no matching row is found for the drug_id in both columns
            drug_info_dict[drug_id] = "Name not found"

print(len(drug_info_dict))

drug_info_dict["CID000004635"]

In [ ]:
drug_info_dict["CID000004635"]

Incorporated SMILES chemical string for 645 unique drugs

In [ ]:
drug_id_to_smiles = {}

for drug_id in lst:
    smiles_string = smiles_df.loc[smiles_df["STITCH"] == drug_id, 'SMILES'].values[0]
    drug_id_to_smiles[drug_id] = smiles_string
    
drug_id_to_smiles
len(drug_id_to_smiles)

In [ ]:
# drug_id_to_smiles["CID000003345"]
drug_id_to_smiles["CID000004635"] # oxycodone

### Model Training Strategies 



#### Strategy: Frozen Layer Vector of ChemBERTa
https://huggingface.co/DeepChem/ChemBERTa-77M-MLM (Developed by DeepChem/ChemBERTa-77M-MLM Transformer Encoder model trained on a large corpus of molecular data 77M molecules)
* DeepChem/ChemBERTa-77M-MLM: Trained using masked language modeling, where parts of the input are masked and the model learns to predict the masked parts.

Step 1: Generate ChemBERTa Features for Each Drug (645 unique drugs)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# uses Byte-Pair Encoding (BPE), such as WordPiece or SentencePiece https://deepchem.readthedocs.io/en/2.4.0/api_reference/tokenizers.html
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
tokenizer_config = tokenizer.init_kwargs

# tokenizer type
print(f"Tokenizer Type: {tokenizer.__class__.__name__}")
print(f"Tokenizer Configuration: {tokenizer_config}")
print(f"Sample Vocabulary Tokens: {list(tokenizer.get_vocab().keys())[:100]}")

# tokenizer converts SMILES strings to token IDs.
model = AutoModel.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
# Example SMILES string for Oxycodone
smiles_oxycodone = "CN1CCC23C4C(=O)CCC2(C1CC5=C3C(=C(C=C5)OC)O4)O"

# Tokenize the SMILES string
tokens = tokenizer.tokenize(smiles_oxycodone)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")

In [ ]:
# Model details
for param in model.parameters():
    param.requires_grad = False

print("Model Layers:")
for i, layer in enumerate(model.encoder.layer):
    print(f"Layer {i}: {layer}")

print("\nModel Parameters:")
for name, param in model.named_parameters():
    print(f"Parameter name: {name}, shape: {param.shape}")
# print(model.summary())
print("The hidden size (vector size) of the model is:", model.config.hidden_size) # 384 is vector size 

# ChemBerta model is frozen, weights are not updated, we only extract features.  
# Extracts the first token from the last hidden state, which serves as a fixed-size feature vector.

def get_chemberta_features(smiles):
    inputs = tokenizer(smiles, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    features = outputs.last_hidden_state[:, 0, :].squeeze() # extracting the last_hidden_state layer (final encoder layer)
    # output fixed size feature vector 384, output of the final encoder layer in the transformer model encoder.layer.2 (index sarts from 0)
    # It contains the embeddings for each token in the input sequence after processing through all layers of the transformer model
    features = features.cpu().numpy()
    return features

#  Store as dictionary where each drug ID maps to its corresponding ChemBERTa feature vector
chemberta_features = {drug_id: get_chemberta_features(smiles) for drug_id, smiles in drug_id_to_smiles.items()}

In [ ]:
drug_info_dict["CID000002173"] #CID000002173  is 'ampicillin'

In [ ]:
#print(drug_id_to_smiles) #{'CID000002173': 'CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O)O)C',}
print(get_chemberta_features('CID000002173'))

**Step 2: DeepChemBERTaDDI (ChemBERTa, PCA  mono side effects)**
* 99 % Variance for mono

In [ ]:
#------------------------------------------------------------------------
# construct SMILES drug features + original mono
se_mono=[]
for k in se2name_mono:
    se_mono.append(k)

print(len(se_mono)) # 10,184 unique mono side effects 

drug_label=np.zeros((n_drugs,len(se_mono)))
for key,value in stitch2se.items():
    j=lst.index(key)
    for v in value:
        i=se_mono.index(v)
        drug_label[j,i]=1
print("Shape of drug_label matrix:", drug_label.shape) # drug_label (645 x 10,184) dimensions before PCA
print("Size of the first row of drug_label matrix:", drug_label[0].size)  # 10184 

# Mono Side Effects Features
pca = PCA(.99) #PCA with n_components=0.95, we are keeping components to retain 95% of the variance dataset's feature space that maximizes the variance (spread) of the data along that direction
pca.fit(drug_label)
drug_label_pca = pca.transform(drug_label) # 99% variance (645, 595)
print("Shape of drug_label_pca matrix:", drug_label_pca.shape)

# ChemBERTa matrix
chemberta_feature_matrix = np.array([chemberta_features[drug_id] for drug_id in lst]) # (645, 384)

# Concatenate ChemBERTa features, PCA-transformed drug-mono features
drug_feat = np.concatenate((chemberta_feature_matrix, drug_label_pca), axis=1)
print("Shape of drug_feat matrix:", drug_feat.shape) # (645, 979)
print(drug_feat[0].size)

## **3. Represent Drug Pair feature by Sum Gate**


### **Sums the features of two drugs for each pair in the training, validation, and test sets.**
* for i in val[k]:
    v.append(drug_feat[i[0]] + drug_feat[i[1]])
* for i in test[k]:
    te.append(drug_feat[i[0]] + drug_feat[i[1]])
* for i in train[k]:
    tr.append(drug_feat[i[0]] + drug_feat[i[1]])

In [ ]:
# construct train & validation & test sets
val_sets=[]
val_labels=[]
for k in range(n_drugdrug_rel_types):
    v=[]
    a=[]
    for  i in val[k]:
        v.append(drug_feat[i[0]]+drug_feat[i[1]])
        a.append(drug_drug_adj_list[k][i[0],i[1]])
    val_sets.append(v)
    val_labels.append(a)
test_sets=[]
test_labels=[]
for k in range(n_drugdrug_rel_types):
    te=[]
    a=[]
    for  i in test[k]:
        te.append(drug_feat[i[0]]+drug_feat[i[1]])
        a.append(drug_drug_adj_list[k][i[0],i[1]])
    test_sets.append(te)
    test_labels.append(a)
train_sets=[]
train_labels=[]
for k in range(n_drugdrug_rel_types):
    tr=[]
    a=[]
    for  i in train[k]:
        tr.append(drug_feat[i[0]]+drug_feat[i[1]])
        a.append(drug_drug_adj_list[k][i[0],i[1]])
    train_sets.append(tr)
    train_labels.append(a)

# constructs the training, validation, and test sets by combining features of drug pairs
val_org=[]
val_label_org=[]
test_org=[]
test_label_org=[]
train_org=[]
train_label_org=[]
for k in range(n_drugdrug_rel_types):
    val_org.append(np.array(val_sets[k]))
    val_label_org.append(np.array(val_labels[k]))
    test_org.append(np.array(test_sets[k]))
    test_label_org.append(np.array(test_labels[k]))
    train_org.append(np.array(train_sets[k]))
    train_label_org.append(np.array(train_labels[k]))

In [ ]:
# number of drug pairs for each set
print("Number of validation pairs:", sum(len(val[k]) for k in range(n_drugdrug_rel_types)))
print("Number of test pairs:", sum(len(test[k]) for k in range(n_drugdrug_rel_types)))
print("Number of training pairs:", sum(len(train[k]) for k in range(n_drugdrug_rel_types)))

In [ ]:
drug_feat.shape[1] # neurons in input layer

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping


model = Sequential()
model.add(Dense(250, input_dim=drug_feat.shape[1], kernel_initializer="glorot_normal"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(300, kernel_initializer="glorot_normal"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(200, kernel_initializer="glorot_normal"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(1, kernel_initializer="glorot_normal", activation="sigmoid"))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

### **5-Fold Cross Validation (5-fold CV) + Early Stopping + Loss**

In [18]:
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping

roc_score = []
aupr_score = []
f_score = []
thr = []
precision = []
recall = []
tpos = []
fpos = []
tneg = []
fneg = []
acc = []
mcc = []
performance_metrics = []
loss_values = []
# for overall ROC curve
all_fpr = []
all_tpr = []

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,        
    verbose=1,
    restore_best_weights=True
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for k in range(n_drugdrug_rel_types):
    print(f"Running for Relationship Type {k}")
    print(f"Total Number of Relationship Types: {n_drugdrug_rel_types}")

    fold_roc_scores = []
    fold_aupr_scores = []
    fold_f_scores = []
    fold_precisions = []
    fold_recalls = []
    fold_accs = []
    fold_mccs = []
    fold_losses = []

    for train_index, val_index in kf.split(train_org[k]):
        X_train, X_val = train_org[k][train_index], train_org[k][val_index]
        y_train, y_val = train_label_org[k][train_index], train_label_org[k][val_index]

        model.fit(X_train, y_train, batch_size=1024, epochs=50, 
                  validation_data=(X_val, y_val), callbacks=[early_stopping])

        loss, accuracy = model.evaluate(test_org[k], test_label_org[k])
        fold_losses.append(loss)

        # Predict probabilities on the test set
        y_probs = model.predict(test_org[k])

        roc = metrics.roc_auc_score(test_label_org[k], y_probs)
        aupr = metrics.average_precision_score(test_label_org[k], y_probs)
        fpr, tpr, thresholds = metrics.roc_curve(test_label_org[k], y_probs)
        scores = [metrics.f1_score(test_label_org[k], to_labels(y_probs, t)) for t in thresholds]
        ma = max(scores)
        idx = np.argmax(scores)
        bt = thresholds[idx]
        p = metrics.precision_score(test_label_org[k], to_labels(y_probs, bt))
        r = metrics.recall_score(test_label_org[k], to_labels(y_probs, bt))

        TP, FP, TN, FN = perf_measure(test_label_org[k], to_labels(y_probs, bt))
        ac = float(TP + TN) / (TP + FP + FN + TN)
        mc = metrics.matthews_corrcoef(test_label_org[k], to_labels(y_probs, bt))

        fold_roc_scores.append(roc)
        fold_aupr_scores.append(aupr)
        fold_f_scores.append(ma)
        fold_precisions.append(p)
        fold_recalls.append(r)
        fold_accs.append(ac)
        fold_mccs.append(mc)

    roc_score.append(np.mean(fold_roc_scores))
    aupr_score.append(np.mean(fold_aupr_scores))
    f_score.append(np.mean(fold_f_scores))
    precision.append(np.mean(fold_precisions))
    recall.append(np.mean(fold_recalls))
    acc.append(np.mean(fold_accs))
    mcc.append(np.mean(fold_mccs))
    loss_values.append(np.mean(fold_losses))

    metrics_dict = {
        "Relationship Type": k,
        "ROC Score": np.mean(fold_roc_scores),
        "AUPR Score": np.mean(fold_aupr_scores),
        "F1 Score": np.mean(fold_f_scores),
        "Precision": np.mean(fold_precisions),
        "Recall": np.mean(fold_recalls),
        "Accuracy": np.mean(fold_accs),
        "MCC": np.mean(fold_mccs),
        "Loss": np.mean(fold_losses)
    }
    performance_metrics.append(metrics_dict)

average_loss = np.mean(loss_values)
print(f"Average Loss: {average_loss}")
print(f"Average ROC Score: {np.mean(roc_score)}")
print(f"Average AUPR Score: {np.mean(aupr_score)}")
print(f"Average F1 Score: {np.mean(f_score)}")
print(f"Average Precision: {np.mean(precision)}")
print(f"Average Recall: {np.mean(recall)}")
print(f"Average Accuracy: {np.mean(acc)}")
print(f"Average MCC: {np.mean(mcc)}")

Running for Relationship Type 0
Total Number of Relationship Types: 964
Epoch 1/50
20/20 [==============================] - 3s 105ms/step - loss: 0.6153 - accuracy: 0.6634 - val_loss: 0.4837 - val_accuracy: 0.7616
Epoch 2/50
20/20 [==============================] - 0s 22ms/step - loss: 0.4641 - accuracy: 0.7844 - val_loss: 0.4352 - val_accuracy: 0.8023
Epoch 3/50
20/20 [==============================] - 0s 21ms/step - loss: 0.4194 - accuracy: 0.8104 - val_loss: 0.4190 - val_accuracy: 0.8157
Epoch 4/50
20/20 [==============================] - 0s 22ms/step - loss: 0.3894 - accuracy: 0.8266 - val_loss: 0.4125 - val_accuracy: 0.8168
Epoch 5/50
20/20 [==============================] - 1s 57ms/step - loss: 0.3620 - accuracy: 0.8402 - val_loss: 0.4052 - val_accuracy: 0.8208
Epoch 6/50
20/20 [============================

In [ ]:
mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.mean(np.array([np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(all_fpr, all_tpr)]), axis=0)
mean_tpr[-1] = 1.0
mean_auc = metrics.auc(mean_fpr, mean_tpr)

plt.figure()
plt.plot(mean_fpr, mean_tpr, color='b', label=f'Mean ROC curve (area = {mean_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Overall Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('overall_roc_curve.png')

plt.show()

##### Sample Code with Early Stopping (without 5-fold CV)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
import numpy as np
import joblib


roc_score = []
aupr_score = []
f_score = []
thr = []
precision = []
recall = []
tpos = []
fpos = []
tneg = []
fneg = []
acc = []
mcc = []
performance_metrics = []
loss_values = []

# for overall ROC curve
all_fpr = []
all_tpr = []

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,        
    verbose=1,
    restore_best_weights=True
)

for k in range(n_drugdrug_rel_types):
    
    print(f"Running for Relationship Type {k}")
    print(f"Total Number of Relationship Types: {n_drugdrug_rel_types}")
    

    model.fit(train_org[k], train_label_org[k], batch_size=1024, epochs=50, 
              validation_data=(val_org[k], val_label_org[k]),  # Included validation data for early stopping
              callbacks=[early_stopping])
    
    loss, accuracy = model.evaluate(test_org[k], test_label_org[k])
    loss_values.append(loss)
    
    # Predict probabilities on the test set
    y_probs = model.predict(test_org[k])

    roc = metrics.roc_auc_score(test_label_org[k], y_probs)
    aupr = metrics.average_precision_score(test_label_org[k], y_probs)
    fpr, tpr, thresholds = metrics.roc_curve(test_label_org[k], y_probs)
    scores = [metrics.f1_score(test_label_org[k], to_labels(y_probs, t)) for t in thresholds]
    ma = max(scores)
    idx = np.argmax(scores)
    bt = thresholds[idx]
    p = metrics.precision_score(test_label_org[k], to_labels(y_probs, bt))
    r = metrics.recall_score(test_label_org[k], to_labels(y_probs, bt))
    
    TP, FP, TN, FN = perf_measure(test_label_org[k], to_labels(y_probs, bt))
    ac = float(TP + TN) / (TP + FP + FN + TN)
    mc = metrics.matthews_corrcoef(test_label_org[k], to_labels(y_probs, bt))

    roc_score.append(roc)
    aupr_score.append(aupr)
    f_score.append(ma)
    thr.append(bt)
    precision.append(p)
    recall.append(r)
    tpos.append(TP)
    fpos.append(FP)
    tneg.append(TN)
    fneg.append(FN)
    acc.append(ac)
    mcc.append(mc)

    metrics_dict = {
        "Relationship Type": k,
        "ROC Score": roc,
        "AUPR Score": aupr,
        "F1 Score": ma,
        "Threshold": bt,
        "Precision": p,
        "Recall": r,
        "True Positives": TP,
        "False Positives": FP,
        "True Negatives": TN,
        "False Negatives": FN,
        "Accuracy": ac,
        "MCC": mc,
        "Loss": loss
    }
    performance_metrics.append(metrics_dict)
    all_fpr.append(fpr)
    all_tpr.append(tpr)

average_loss = np.mean(loss_values)
print(f"Average Loss: {average_loss}")
print(f"Average ROC Score: {np.mean(roc_score)}")
print(f"Average AUPR Score: {np.mean(aupr_score)}")
print(f"Average F1 Score: {np.mean(f_score)}")
print(f"Average Threshold: {np.mean(thr)}")
print(f"Average Precision: {np.mean(precision)}")
print(f"Average Recall: {np.mean(recall)}")
print(f"Average Accuracy: {np.mean(acc)}")
print(f"Average MCC: {np.mean(mcc)}")

Write Output Dataframe to workspace blob storage

In [ ]:
import pandas as pd

# Add a mapping of relationship types to side effect names
side_effect_mapping = {i: common_se_names[i] for i in range(n_drugdrug_rel_types)}
df_performance = pd.DataFrame(performance_metrics)
df_performance['Side Effect Name'] = df_performance['Relationship Type'].map(side_effect_mapping)
df_performance


In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
import time
import pandas as pd
from azureml.core import Workspace, Datastore

# Load the workspace config
workspace = Workspace.from_config()
default_datastore = workspace.get_default_datastore()

# Save the DataFrame to a CSV file
csv_file_path = "Results_ChemBERTa77M_mono_PCA99.csv"
df_performance.to_csv(csv_file_path, index=False)
default_datastore.upload_files([csv_file_path], target_path=".", overwrite=True, show_progress=True)

v2 = "cleaned" + time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())
my_path = f"./{csv_file_path}"

# Define the data asset
my_data = Data(
    name="Results_ChemBERTa77M_mono_PCA99",
    version=v2,
    description="Results_ChemBERTa77M_mono_PCA99",
    tags={"output_data": "true", "format": "csv"},
    path=my_path,
    type=AssetTypes.URI_FILE,
)

# Create or update the data asset
my_data = ml_client.data.create_or_update(my_data)
print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")